In [78]:
# MAIN 
from sage.modules.free_module_integer import IntegerLattice
from generate import *
from closest_vs_minimal import shortest_lc_in_cube

def generate_new_examples(iterations, dimension, perimeter, jsonfilename, printing = False, functioning = True) -> None:
    '''
    generates >iterations< of >dis/functioning< matrices and saves them in a json file.
    returns number of suitable cases
    '''
    output_data = []
    i = 0
    while i < iterations:
    # for i in range(iterations):
        B = randomMatrix(dimension, perimeter)
        G = gram_matrix(B)

        L = IntegerLattice(B, lll_reduce=False)

        sv_LLL = shortestVector(B.LLL())
        lcLLL = B.solve_left(sv_LLL)

        sv_exact = L.shortest_vector()
        lc_exact = B.solve_left(sv_exact)
        sv_exact_norm = sv_exact.norm().n()

        if (1 or -1) not in lc_exact.list():
            continue


        # Compare the result with solution of the cube algorithm
        this_case_works, lcCube = compare_with_cube(lc_exact,B, G)
        if this_case_works == functioning:
            case_info = into_dict(B, lcLLL, lcCube, lc_exact)
            output_data.append(case_info)
        i +=1
    
    # Format output data
    if printing: print_listdict(output_data)
    dict = format_data(output_data)
    into_json(dict, jsonfilename)
    return len(output_data)


def find_indices(lst, targets):
    return [i for i, x in enumerate(lst) if x in targets]


def compare_with_cube(lc_exact, B,G) -> bool:
    for current_row in find_indices(lc_exact, [1,-1]):
        exact_component = lc_exact[current_row] # 1 or -1
        
        # however we only insert 1, otherwise its messy
        lcCube = find_real_minimum(G, current_row, 1) 
        lc_shortest_in_cube =  vector(shortest_lc_in_cube(lcCube, B))

        # Check for invalid cases
        if norm_G(lcCube, G) > norm_G(lc_exact, G) or lcCube == zero_vector(SR, dimension):
            print("invalid case")
            continue
        
        # Check if the two results are the same 
        if lc_shortest_in_cube == lc_exact or lc_shortest_in_cube == (-1)*lc_exact:
            return True, lcCube
    return False, lcCube

        
def find_real_minimum(G, current_row, unit_component) -> vector:
    """
    returns linear combination of the minimum over real numbers
    the >current_row< parameter is the coordinate of fixed col/row idk
    the >K< parameter is the number inserted into this column
    LLL component is +- 1
    """
    matrixA = matrix(dimension - 1, dimension - 1, 0) # square matrix of size (dimension - 1) x (dimension - 1), filled with zeros.
    matrixB = matrix(dimension - 1, 1, 0) # column matrix of size (dimension - 1) x 1, filled with zeros.
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(dimension):
        if row != current_row:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != current_row]
            matrixB[b] = sum([unit_component * G[row,j] for j in range(len(G[row])) if j == current_row])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right((-1)*unit_component * matrixB)).list()
    result.insert(current_row, unit_component)
    return vector(result).n(digits=5)

In [77]:
dimension = 3

perimeter = 1000
sensitivity = 1
tries = 1
# jsonfilename = "functioning-false-dim-3-randomperimeter-1000.json"
jsonfilename = "dummyfile.json"
generate_new_examples(tries, dimension, perimeter, jsonfilename, printing=True, functioning=False)

[-433 -843 -167]
[-696 -909  841]
[ 118  142 -298] :  B
[ 926027  927208 -121034]
[ 927208 2017978 -461824]
[-121034 -461824  122892] :  G
(-1, 2, 7) :  lincomb_exact
(-1, 2, 7) :  lincomb_LLL
(1.0000, -1.6723, -5.2997) :  lincomb_cube
(1, -2, -6) :  lincomb_cube_shortest
272.43 :  exact.norm
272.43 :  LLL.norm
286.10 :  shortest_cube.norm
(-133, 19, -237) :  sv_LLL
(-133, 19, -237) :  sv_exact
(251, 123, -61) :  sv_shortst_cube



1